In [1]:
from DataGenerator import TimeSeriesGenerator, suddenDrift, incrementalDrift
from conditions import scenarios
from sklearn.preprocessing import StandardScaler
from grouped_series import SeriesGrouper, ExperimentTracker, Experiment
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.pipeline import Pipeline
from TimeSeriesGradientBoosting import TimeSeriesGradientBoosting
from xgboost import XGBRegressor
from sklearn.linear_model import SGDRegressor
from sklearn.model_selection import train_test_split

In [2]:
onehot_cols = ColumnTransformer([('one_hot_encoder', OneHotEncoder(
), make_column_selector(dtype_include=object))], remainder=StandardScaler())

xgb_pipe = Pipeline([
    ('preprocessor', onehot_cols),
    ('regressor', XGBRegressor())
])

sgd_pipe = Pipeline([
    ('preprocessor', onehot_cols),
    ('regressor', SGDRegressor())
])

In [3]:
def full_run():
    products = []
    customers = []
    for i in range(2):
        product = TimeSeriesGenerator(size=365, amountOfVariables=7)
        customer = TimeSeriesGenerator(size=365, amountOfVariables=3)
        products.append(product)
        customers.append(customer)
    thesis = ExperimentTracker(products, customers, scenarios())
    thesis.runExperiment(algorithm=sgd_pipe)
    return thesis


In [4]:
thesis = full_run()

Finished experiment! Elapsed time: 0.11671328544616699, total Elapsed time: 0.11671328544616699, Type: Sudden Drift, Dropped variables: None, magnitude: Small, Drift time: Fully observed, importance: Important
Finished experiment! Elapsed time: 0.08579277992248535, total Elapsed time: 0.20350313186645508, Type: Sudden Drift, Dropped variables: None, magnitude: Small, Drift time: Half observed, importance: Important
Finished experiment! Elapsed time: 0.0718240737915039, total Elapsed time: 0.275327205657959, Type: Sudden Drift, Dropped variables: None, magnitude: Small, Drift time: Unobserved, importance: Important
Finished experiment! Elapsed time: 0.09097051620483398, total Elapsed time: 0.36629772186279297, Type: Sudden Drift, Dropped variables: None, magnitude: Small, Drift time: Fully observed, importance: Medium
Finished experiment! Elapsed time: 0.08279776573181152, total Elapsed time: 0.4490954875946045, Type: Sudden Drift, Dropped variables: None, magnitude: Small, Drift time: 

In [5]:
results = thesis.resultsToDF()

In [6]:
results[["SMAPE", "Drift type", "Drift time", "Drift magnitude"]].groupby(["Drift type", "Drift time", "Drift magnitude"]).aggregate(["mean", "std", "min", "max"])

SMAPE                  \
                                                       mean       std   min   
Drift type        Drift time     Drift magnitude                              
Incremental Drift Fully observed Large             3.336667  3.229093  1.74   
                                 Small             2.413333  0.580992  2.10   
                  Half observed  Large            12.340000  6.641391  3.15   
                                 Small             4.035000  1.560824  2.25   
                  Unobserved     Large             9.514167  4.953617  3.12   
                                 Small             3.645000  1.165687  2.35   
No Drift          No Drift       No Drift          2.285000  0.017321  2.26   
Sudden Drift      Fully observed Large             3.425833  3.450132  1.74   
                                 Small             2.434167  0.644268  2.09   
                  Half observed  Large             4.130000  4.782424  1.96   
                                 Small             2.544167  0.799630  2.21   
                  Unobserved     Large            13.700000  7.576070  3.06   
                                 Small             4.280833  1.907156  2.18   
Sudden Shock      Noise          Large             2.465000  0.330101  2.29   
                                 Small             2.321250  0.071402  2.27   

                                                         
                                                    max  
Drift type        Drift time     Drift magnitude         
Incremental Drift Fully observed Large            12.59  
                                 Small             4.19  
                  Half observed  Large            21.84  
                                 Small             6.23  
                  Unobserved     Large            17.74  
                                 Small             5.50  
No Drift          No Drift       No Drift          2.30  
Sudden Drift      Fully observed Large            13.30  
                                 Small             4.38  
                  Half observed  Large            18.09  
                                 Small             5.06  
                  Unobserved     Large            24.86  
                                 Small             7.25  
Sudden Shock      Noise          Large             2.96  
                                 Small             2.45

In [7]:
results.sort_values("SMAPE", ascending=False)   

,Dropped variable,Drift type,Drift magnitude,Variable importance,Drift time,MSE,SMAPE
142,6.0,Incremental Drift,Large,Medium,Half observed,162413.496101,[28.51]
68,0.0,Incremental Drift,Large,Medium,Half observed,69922.463470,[21.28]
143,6.0,Incremental Drift,Large,Medium,Unobserved,120920.732804,[19.91]
31,NaN,Incremental Drift,Large,Medium,Half observed,50532.583939,[19.45]
69,0.0,Incremental Drift,Large,Medium,Unobserved,90685.703338,[18.55]
...,...,...,...,...,...,...,...
27,NaN,Incremental Drift,Large,Important,Fully observed,14.963985,[2.86]
83,3.0,Sudden Drift,Large,Important,Fully observed,14.710612,[2.84]
9,NaN,Sudden Drift,Large,Important,Fully observed,14.815449,[2.84]
138,6.0,Incremental Drift,Large,Important,Fully observed,14.940066,[2.84]
